<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install unidecode
!pip install word2number
!pip install contractions
!python -m spacy download en_core_web_md 

     |████████████████████████████████| 96.4 MB 1.3 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [17]:
!python -m spacy download en

     |████████████████████████████████| 12.0 MB 5.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [18]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
import gensim
from gensim.models import Word2Vec
import string
string.punctuation
nltk.download('stopwords')
import re
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import contractions
from nltk.tokenize.treebank import TreebankWordDetokenizer
# load spacy model, can be "en_core_web_sm" as well
from sklearn.model_selection import train_test_split
import xgboost as xgb



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
df_fake = pd.read_csv('/content/drive/MyDrive/gossipcop_fake.csv')
df_real = pd.read_csv('/content/drive/MyDrive/gossipcop_real.csv')
df_fake['labelML']=0
df_real['labelML']=1
df= df_fake.head(50).append(df_real.head(50))
df.reset_index(drop=True, inplace=True)

In [26]:
df['tweet_count'] = df['tweet_ids'].str.split('\t').str.len()

In [27]:
!pip3 install spacy
import spacy
spacy.load('en')

In [28]:
nlp = spacy.load('en_core_web_md')

# exclude words from spacy stopwords list
deselect_stop_words = ['no', 'not']
for w in deselect_stop_words:
    nlp.vocab[w].is_stop = False


def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text


def remove_whitespace(text):
    """remove extra whitespaces from text"""
    text = text.strip()
    return " ".join(text.split())


def remove_accented_chars(text):
    """remove accented characters from text, e.g. café"""
    text = unidecode.unidecode(text)
    return text


def expand_contractions(text):
    """expand shortened words, e.g. don't to do not"""
    text = contractions.fix(text)
    return text


def text_preprocessing(text, accented_chars=True, contractions=True, 
                       convert_num=True, extra_whitespace=True, 
                       lemmatization=True, lowercase=True, punctuations=True,
                       remove_html=True, remove_num=True, special_chars=True, 
                       stop_words=True):
    """preprocess text with default option set to true for all steps"""
    if remove_html == True: #remove html tags
        text = strip_html_tags(text)
    if extra_whitespace == True: #remove extra whitespaces
        text = remove_whitespace(text)
    if accented_chars == True: #remove accented characters
        text = remove_accented_chars(text)
    if contractions == True: #expand contractions
        text = expand_contractions(text)
    if lowercase == True: #convert all characters to lowercase
        text = text.lower()

    doc = nlp(text) #tokenise text

    clean_text = []
    
    for token in doc:
        flag = True
        edit = token.text
        # remove stop words
        if stop_words == True and token.is_stop and token.pos_ != 'NUM': 
            flag = False
        # remove punctuations
        if punctuations == True and token.pos_ == 'PUNCT' and flag == True: 
            flag = False
        # remove special characters
        if special_chars == True and token.pos_ == 'SYM' and flag == True: 
            flag = False
        # remove numbers
        if remove_num == True and (token.pos_ == 'NUM' or token.text.isnumeric()) \
        and flag == True:
            flag = False
        # convert number words to numeric numbers
        if convert_num == True and token.pos_ == 'NUM' and flag == True:
            edit = w2n.word_to_num(token.text)
        # convert tokens to base form
        elif lemmatization == True and token.lemma_ != "-PRON-" and flag == True:
            edit = token.lemma_
        # append tokens edited and not removed to list 
        if edit != "" and flag == True:
            clean_text.append(edit)        
    return TreebankWordDetokenizer().detokenize(clean_text)

In [29]:
df['title_preprocessed'] = df['title'].apply(text_preprocessing)

In [30]:
s  = df['title_preprocessed'].str.split(' ').explode().to_frame()

cols = s['title_preprocessed'].drop_duplicates(keep='first').tolist()

df2 = pd.concat([df, pd.crosstab(s.index, s["title_preprocessed"])[cols]], axis=1).replace(
    {1: 1, 0: 0}
)
print(df2.columns)

df2= df2.drop(['id','news_url','title','tweet_ids','title_preprocessed'],axis=1)

Index(['id', 'news_url', 'title', 'tweet_ids', 'labelML', 'tweet_count',
       'title_preprocessed', 'miley', 'cyrus', 'liam',
       ...
       'iggy', 'azalea', 'f', 'moron', 'quavo', 'delilah', 'son', 'suicide',
       'heartbreaking', 'goodbye'],
      dtype='object', length=543)


In [31]:
df2.reset_index(drop=True, inplace=True)
X, y = df2.iloc[:, 1:], df2['labelML']

# Create the training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train, y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds == y_test)) / y_test.shape[0]
print("accuracy: %f" % (accuracy))

accuracy: 0.800000
